In [1]:
import urllib3
from bs4 import BeautifulSoup
import requests
import certifi
import time
import datetime

/Users/xu/anaconda/lib/python2.7/site-packages/cffi/model.py:532: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


In [2]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
#                      passwd="root",  # your password
                     db="pipa_db")        # name of the data base

cur = db.cursor()

In [12]:
dailyMax = 1000
batchNum = 20
sleeptimeforPubmed =5
backstep =20

startingPMID=28304400 
if cur.execute('select pmid from last_pmid') !=0:
    endingPMIDtuple = max(cur.fetchall())
    endingPMID = endingPMIDtuple[0]
    startingPMID=endingPMID -backstep

print startingPMID
    

28304400


In [13]:
cur.execute('truncate articles')
# erase data or re-create articles table

0L

In [14]:
%%time
        for i in range (startingPMID, startingPMID+dailyMax, batchNum):
            PMIDs = ""

            for j in range (batchNum):
                PMIDs = PMIDs + str(i + j ) +","

            url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text" 

            #time.sleep(sleeptimeforPubmed)
            http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
            r = http.request('GET', url)
            soup = BeautifulSoup(r.data, "lxml")

            allInfo = unicode(soup.pre.string)

            if allInfo.find('PMID- ')<0:
                print url
                print 'bad end'        
                break

            while allInfo.find('AB  -')>0:
                IdIndexStart = allInfo.index('PMID- ')+6
                IdIndexEnd = allInfo.index('\n', IdIndexStart)
                PMID = allInfo[IdIndexStart:IdIndexEnd]

                tiIndexStart = allInfo.index('TI  -')+6
                tiIndexEnd =   allInfo.index('PG  -', tiIndexStart)-1
                ti = allInfo[tiIndexStart:tiIndexEnd]
                
                abIndexStart = allInfo.index('AB  -')+6
                abIndexEnd =   allInfo.index('FAU -', abIndexStart)-1
                ab = ti+"        \nAbstract:"+allInfo[abIndexStart:abIndexEnd]

                query = "insert into articles (pmid, abstract) values ("+ PMID +", '"+ ab + "')"
                endingPMID = int(PMID)
                if (endingPMID%10==0):
                    print endingPMID

                try:
                    cur.execute(query)
                    db.commit()

                except:
                    db.rollback()

                allInfo = allInfo[abIndexEnd:]

        now = datetime.datetime.now()       
        query = 'insert into last_pmid (record_date, pmid) values ("%s", %d)'%(now, endingPMID)
        cur.execute(query)
        db.commit()
# write endingPMID to db
db.close()    
    

28304400
28304410
28304420
28304440
28304450
28304460
28304470
28304480
28304490
28304500
28304520
28304530
28304540
28304550
28304560
28304570
28304580
28304590
28304600
28304610
28304620
28304630
28304640
28304650
28304660
28304670
28304680
28304690
28304700
28304710
28304720
28304730
28304740
28304750
28304760
28304770
28304780
28304790
28304800
28304810
28304820
28304830
28304840
28304850
28304860
28304870
28304880
28304890
28304900
28304910
28304920
28304930
28304940
28304950
28304960
28304980
28304990
28305000
28305010
28305020
28305030
28305040
28305050
28305060
28305070
28305080
28305090
28305100
28305110
28305120
28305130
28305140
28305150
28305160
28305180
28305190
28305200
28305210
28305220
28305230
28305240
28305250
28305260
28305270
28305280
28305290
28305300
28305310
28305320
28305330
28305340
28305350
28305360
28305370
28305380
28305390
CPU times: user 1.04 s, sys: 117 ms, total: 1.16 s
Wall time: 44 s
